In [13]:
import support_functions as sf

sf.generate_very_simple_portfolio_history(no_of_policies=10000, runtime=5)

In [14]:
policy_histories = sf.load_troubled_life_policy_data()
#policy_histories

In [15]:
policy_histories, history_lengths, max_history_length = sf.prepare_troubled_life_policy_data(policy_histories=policy_histories)
#policy_histories


In [16]:
# Extract features and labels from dataset as numpy.ndarray(s)
policy_histories_labels = policy_histories['troubled'].as_matrix()
policy_histories_features = policy_histories[['premium', 'current_capital']].as_matrix()

# Reshape features from (overall_no_of_histories, 2) to (overall_no_of_policies, maximum_history_length, 2)
policy_histories_features = policy_histories_features.reshape((policy_histories.index.levels[0].shape[0], max_history_length, -1))

# Reshape labels from (overall_no_of_histories) to (overall_no_of_policies, maximum_history_length) and take maximum of each row (0 or 1)
policy_histories_labels = policy_histories_labels.reshape(policy_histories.index.levels[0].shape[0], -1).max(axis=1)

history_lengths = history_lengths[:,1]

#policy_histories_features, policy_histories_labels, history_lengths

In [17]:
import tensorflow as tf

tf.reset_default_graph()
tf.set_random_seed(42)

n_steps = max_history_length
n_inputs = 2

n_neurons = 100
n_outputs = 2

learning_rate = 0.001

# Placeholders for our input sequences
seq_length = tf.placeholder(tf.int32, [None])
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.int32, [None])

# A single RNN layer of n_neurons neurons
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.nn.relu)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32, sequence_length=seq_length)

# We feed the layer's output of the last timestep into a fully connected layer of 2 neurons
# (1 per class, troubled or not). Softmax layer is next
logits = tf.layers.dense(states, n_outputs)
y_pred = tf.argmax(tf.nn.softmax(logits), axis=1)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
#xentropy = y - y_pred
#xentropy = tf.abs(xentropy)

# Loss function and Adam Optimizer
loss = tf.reduce_mean(tf.cast(xentropy, tf.float32))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)

correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()


In [19]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

n_epochs = 100
batch_size = 1000

X_batch = policy_histories_features
y_batch = policy_histories_labels
seq_length_batch = history_lengths

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for i in range(policy_histories_labels.shape[0] // batch_size):
            X_batch = policy_histories_features[i * batch_size : (i + 1) * batch_size]
            y_batch = policy_histories_labels[i * batch_size : (i + 1) * batch_size]
            seq_length_batch = history_lengths[i * batch_size : (i + 1) * batch_size]
            
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, seq_length: seq_length_batch})
            #print(outputs.eval(feed_dict={X: X_batch, y: y_batch, seq_length: seq_length_batch}))
        #print(logits.eval(feed_dict={X: X_batch, y: y_batch, seq_length: seq_length_batch}))
        y_batch_pred = y_pred.eval(feed_dict={X: X_batch, y: y_batch, seq_length: seq_length_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch, seq_length: seq_length_batch})
        #acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
        print(epoch, "Train accuracy:", acc_train)#, "Test accuracy:", acc_test)
        print(loss.eval(feed_dict={X: X_batch, y: y_batch, seq_length: seq_length_batch}))
        print(confusion_matrix(y_batch, y_batch_pred))
        print(classification_report(y_batch, y_batch_pred, target_names=['ok', 'troubled']))




0 Train accuracy: 0.776
358.37
[[776   0]
 [224   0]]
             precision    recall  f1-score   support

         ok       0.78      1.00      0.87       776
   troubled       0.00      0.00      0.00       224

avg / total       0.60      0.78      0.68      1000

1 Train accuracy: 0.776
50.6838
[[776   0]
 [224   0]]
             precision    recall  f1-score   support

         ok       0.78      1.00      0.87       776
   troubled       0.00      0.00      0.00       224

avg / total       0.60      0.78      0.68      1000



/Users/gopora/MyStuff/Dev/miniconda3/envs/pie35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


2 Train accuracy: 0.776
29.3997
[[776   0]
 [224   0]]
             precision    recall  f1-score   support

         ok       0.78      1.00      0.87       776
   troubled       0.00      0.00      0.00       224

avg / total       0.60      0.78      0.68      1000

3 Train accuracy: 0.179
25.724
[[  0 776]
 [ 45 179]]
             precision    recall  f1-score   support

         ok       0.00      0.00      0.00       776
   troubled       0.19      0.80      0.30       224

avg / total       0.04      0.18      0.07      1000

4 Train accuracy: 0.866
3.6167
[[776   0]
 [134  90]]
             precision    recall  f1-score   support

         ok       0.85      1.00      0.92       776
   troubled       1.00      0.40      0.57       224

avg / total       0.89      0.87      0.84      1000



5 Train accuracy: 0.866
3.85846
[[776   0]
 [134  90]]
             precision    recall  f1-score   support

         ok       0.85      1.00      0.92       776
   troubled       1.00      0.40      0.57       224

avg / total       0.89      0.87      0.84      1000

6 Train accuracy: 0.868
3.10776
[[776   0]
 [132  92]]
             precision    recall  f1-score   support

         ok       0.85      1.00      0.92       776
   troubled       1.00      0.41      0.58       224

avg / total       0.89      0.87      0.85      1000

7 Train accuracy: 0.867
3.49948
[[776   0]
 [133  91]]
             precision    recall  f1-score   support

         ok       0.85      1.00      0.92       776
   troubled       1.00      0.41      0.58       224

avg / total       0.89      0.87      0.84      1000



8 Train accuracy: 0.868
3.33258
[[776   0]
 [132  92]]
             precision    recall  f1-score   support

         ok       0.85      1.00      0.92       776
   troubled       1.00      0.41      0.58       224

avg / total       0.89      0.87      0.85      1000

9 Train accuracy: 0.868
3.1688
[[776   0]
 [132  92]]
             precision    recall  f1-score   support

         ok       0.85      1.00      0.92       776
   troubled       1.00      0.41      0.58       224

avg / total       0.89      0.87      0.85      1000

10 Train accuracy: 0.868
3.22825
[[776   0]
 [132  92]]
             precision    recall  f1-score   support

         ok       0.85      1.00      0.92       776
   troubled       1.00      0.41      0.58       224

avg / total       0.89      0.87      0.85      1000



...

86 Train accuracy: 1.0
0.00503089
[[776   0]
 [  0 224]]
             precision    recall  f1-score   support

         ok       1.00      1.00      1.00       776
   troubled       1.00      1.00      1.00       224

avg / total       1.00      1.00      1.00      1000

87 Train accuracy: 1.0
0.00495793
[[776   0]
 [  0 224]]
             precision    recall  f1-score   support

         ok       1.00      1.00      1.00       776
   troubled       1.00      1.00      1.00       224

avg / total       1.00      1.00      1.00      1000

88 Train accuracy: 1.0
0.00488637
[[776   0]
 [  0 224]]
             precision    recall  f1-score   support

         ok       1.00      1.00      1.00       776
   troubled       1.00      1.00      1.00       224

avg / total       1.00      1.00      1.00      1000



89 Train accuracy: 1.0
0.00481696
[[776   0]
 [  0 224]]
             precision    recall  f1-score   support

         ok       1.00      1.00      1.00       776
   troubled       1.00      1.00      1.00       224

avg / total       1.00      1.00      1.00      1000

90 Train accuracy: 1.0
0.00474887
[[776   0]
 [  0 224]]
             precision    recall  f1-score   support

         ok       1.00      1.00      1.00       776
   troubled       1.00      1.00      1.00       224

avg / total       1.00      1.00      1.00      1000

91 Train accuracy: 1.0
0.00468215
[[776   0]
 [  0 224]]
             precision    recall  f1-score   support

         ok       1.00      1.00      1.00       776
   troubled       1.00      1.00      1.00       224

avg / total       1.00      1.00      1.00      1000



92 Train accuracy: 1.0
0.00461738
[[776   0]
 [  0 224]]
             precision    recall  f1-score   support

         ok       1.00      1.00      1.00       776
   troubled       1.00      1.00      1.00       224

avg / total       1.00      1.00      1.00      1000

93 Train accuracy: 1.0
0.00455308
[[776   0]
 [  0 224]]
             precision    recall  f1-score   support

         ok       1.00      1.00      1.00       776
   troubled       1.00      1.00      1.00       224

avg / total       1.00      1.00      1.00      1000

94 Train accuracy: 1.0
0.00449156
[[776   0]
 [  0 224]]
             precision    recall  f1-score   support

         ok       1.00      1.00      1.00       776
   troubled       1.00      1.00      1.00       224

avg / total       1.00      1.00      1.00      1000



95 Train accuracy: 1.0
0.00442987
[[776   0]
 [  0 224]]
             precision    recall  f1-score   support

         ok       1.00      1.00      1.00       776
   troubled       1.00      1.00      1.00       224

avg / total       1.00      1.00      1.00      1000

96 Train accuracy: 1.0
0.00437049
[[776   0]
 [  0 224]]
             precision    recall  f1-score   support

         ok       1.00      1.00      1.00       776
   troubled       1.00      1.00      1.00       224

avg / total       1.00      1.00      1.00      1000

97 Train accuracy: 1.0
0.00431163
[[776   0]
 [  0 224]]
             precision    recall  f1-score   support

         ok       1.00      1.00      1.00       776
   troubled       1.00      1.00      1.00       224

avg / total       1.00      1.00      1.00      1000



98 Train accuracy: 1.0
0.004255
[[776   0]
 [  0 224]]
             precision    recall  f1-score   support

         ok       1.00      1.00      1.00       776
   troubled       1.00      1.00      1.00       224

avg / total       1.00      1.00      1.00      1000

99 Train accuracy: 1.0
0.00419869
[[776   0]
 [  0 224]]
             precision    recall  f1-score   support

         ok       1.00      1.00      1.00       776
   troubled       1.00      1.00      1.00       224

avg / total       1.00      1.00      1.00      1000

